In [54]:
from utils import make_need_tree,visualize_tree,make_storage,make_task_list,calc_start_end_time
import plotly.express as px
import pandas as pd
import json
import random
import copy

In [55]:
storage = make_storage()
storage['apple']=10
tasks=make_task_list(['hamburger','bacon and eggs','cheese','baked potato','pancake','pancake'
                      ],storage)
len(tasks)

80

In [56]:
# number of machines
sources={
    'Bakery':1, 'Chicken':2, 'Feed Mill':1, 'Field':8   , 'Raspberry bush':1,'Sugar Mill':1,'Cake Oven':1,
    'Cherry tree':3,'Cow':2,'BBQ Grill':1,'Dairy':1,"Pig":2,'Popcorn Pot':1,'Loom':1,'Sewing Machine':1,
    'Pie Oven':1
}
machines ={}
# make machines is: key is same as sources, value is a list of machine_id
# machine_id must be unique from other machines
count  = 0
for key,value in sources.items():
    machines[key]=list(range(count,count+value))
    count+=value
machines

{'Bakery': [0],
 'Chicken': [1, 2],
 'Feed Mill': [3],
 'Field': [4, 5, 6, 7, 8, 9, 10, 11],
 'Raspberry bush': [12],
 'Sugar Mill': [13],
 'Cake Oven': [14],
 'Cherry tree': [15, 16, 17],
 'Cow': [18, 19],
 'BBQ Grill': [20],
 'Dairy': [21],
 'Pig': [22, 23],
 'Popcorn Pot': [24],
 'Loom': [25],
 'Sewing Machine': [26],
 'Pie Oven': [27]}

In [57]:
import random

def make_chromosome(tasks,machines):
    tasks = copy.deepcopy(tasks)
    random.shuffle(tasks)
    tasks.sort(key=lambda x:x['order'],reverse=True)
    for task in tasks:
        machine_id=random.choice(machines[task['source']])
        task['machine_id']=machine_id
    return tasks

def calc_fitness(chromosome):
    """calculate the fitness of a chromosome
    chromosome: a chromosome
    
    return: a fitness value
    """
    score = 0
    chromosome = calc_start_end_time(chromosome,machines)
    # maximum end_time
    max_end_time = max([task['end_time'] for task in chromosome])
    score += 10000/max_end_time
    for task in chromosome:
        if task['target']:
            score+=1/task['end_time']
    return score
    
def mutation(chromosome):
    """mutate a chromosome
    chromosome: a chromosome
    
    return: a chromosome
    """
    chromosome = copy.deepcopy(chromosome)
    if random.random()<0.5:
        gen = random.choice(chromosome)
        # new_machine_id = random.choice(machines[gen['source']])
        # next id in the same machine
        machine_id = gen['machine_id']
        idx = machines[gen['source']].index(machine_id)
        new_machine_id = machines[gen['source']][(idx+1)%len(machines[gen['source']])]
        gen['machine_id']=new_machine_id
        
        gen1,gen2 = random.sample(chromosome,2)
        idx1 = chromosome.index(gen1)
        idx2 = chromosome.index(gen2)
        if idx1>idx2:
            gen1,gen2=gen2,gen1
        if gen1['order']>=gen2['order']:
            chromosome[idx1],chromosome[idx2]=chromosome[idx2],chromosome[idx1]
    return chromosome


In [58]:
# simple GA algorithm
pop_size = 500
iteration = 50
# generate inital population
population = []
for i in range(pop_size):
    population.append(make_chromosome(tasks,machines))

for i in range(iteration):
    fitness = []
    for chromosome in population:
        fitness.append(calc_fitness(chromosome))
    # generate new population
    new_population = []
    best_chromosome = population[fitness.index(max(fitness))]
    new_population.append(best_chromosome)
    print(f'iteration {i}, best fitness {fitness[0]}, best time {max([task["end_time"] for task in best_chromosome])}')
    for i in range(pop_size-1):
        # no crossover
        # mutation
        new_population.append(mutation(population[i]))
    population = new_population

iteration 0, best fitness 0.20851391807879002, best time 36360
iteration 1, best fitness 0.2753002149638956, best time 36360
iteration 2, best fitness 0.2753002149638956, best time 36360
iteration 3, best fitness 0.27530647345197007, best time 36360
iteration 4, best fitness 0.27537291418646004, best time 36300
iteration 5, best fitness 0.27446604087077564, best time 36300
iteration 6, best fitness 0.2744648014830332, best time 36360
iteration 7, best fitness 0.27529957898144075, best time 36360
iteration 8, best fitness 0.27529957898144075, best time 36360
iteration 9, best fitness 0.27529957898144075, best time 36360
iteration 10, best fitness 0.2753608264659701, best time 36360
iteration 11, best fitness 0.2753608264659701, best time 36360
iteration 12, best fitness 0.275365410228122, best time 36360
iteration 13, best fitness 0.27537267171957697, best time 36360
iteration 14, best fitness 0.27537267171957697, best time 36300
iteration 15, best fitness 0.27574824501731626, best time

In [59]:
# chromosome = make_chromosome(tasks,machines)
chromosome = population[fitness.index(max(fitness))]
test_tasks = calc_start_end_time(chromosome,machines)
df = pd.DataFrame(test_tasks)
# task is source + machine_id
df['task']=df['source']+' '+df['machine_id'].astype(str)
df['start'] = pd.to_datetime('today')+pd.to_timedelta(df['start_time'],unit='s')
df['end'] = pd.to_datetime('today')+pd.to_timedelta(df['end_time'],unit='s')
df.head()

,id,name,dependencies,duration,source,machine_id,target,order,start_time,end_time,task,start,end
0,15,hamburger,"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]",7200,BBQ Grill,20,True,1,17400,24600,BBQ Grill 20,2024-01-04 10:23:30.747676,2024-01-04 12:23:30.748696
1,52,corn,[],300,Field,7,False,5,0,300,Field 7,2024-01-04 05:33:30.747676,2024-01-04 05:38:30.748696
2,43,corn,[],300,Field,8,False,5,0,300,Field 8,2024-01-04 05:33:30.747676,2024-01-04 05:38:30.748696
3,45,soybean,[],1200,Field,10,False,5,0,1200,Field 10,2024-01-04 05:33:30.747676,2024-01-04 05:53:30.748696
4,44,soybean,[],1200,Field,6,False,5,0,1200,Field 6,2024-01-04 05:33:30.747676,2024-01-04 05:53:30.748696


In [60]:
print("Max time: ",max(df['end_time']))
fig = px.timeline(df, x_start="start", x_end="end", y="task", color="name",hover_name='name', 
                    hover_data={'id':True,'duration':True,'start':False,'end':False,'task':False,'name':False,'source':False,'machine_id':False,'dependencies':True})
fig.update_yaxes(autorange="reversed")
fig.show()
#36300

Max time:  35400
